# Path likelihood

In [26]:
using BSON: @save, @load
using Statistics
using DelimitedFiles
using Printf
using MDToolbox
using StatsBase 
using LinearAlgebra
using Random
using Plots

include("../src/afm.jl")

baumwelch (generic function with 1 method)

In [27]:
nq = 576
qs = readdlm("../data/quaternion/QUATERNION_LIST_$(nq)_Orient")
models = load_model("../data/t1r/cluster.pdb");
nmodel = size(models, 1)

test_radius = 25
pred_radii = [15, 18, 20, 25, 30, 32, 35]
sigma_noise = 3
sharpness = 10
nframe = 100

100

In [28]:
seed = MersenneTwister(334)
extracted_qs = sample(seed, 1:576, 50, replace=false);

In [29]:
iq_true = extracted_qs[1]

240

In [30]:
@load "../data/t1r/t1r.bson" T pi_i

In [31]:
@load "data/test_case/radius_$(test_radius)/sharpness_$(sharpness)/q_$(nq)/iq_$(iq_true)_noise_$(sigma_noise)_nframe_$(nframe).bson" afms param imodel_array dxdy_array

In [33]:
my_nframe = 100

logLs = []
for pred_radius in pred_radii
    @load "data/result/test_radius_$(test_radius)/pred_radius_$(pred_radius)/sharpness_$(sharpness)/q_$(nq)/iq_$(iq_true)_noise_$(sigma_noise)_nframe_$(nframe).bson" params r
    logL = zeros(Float64, nq)
    for iq = 1:nq
        ids = []
        for imodel = 1:nmodel
            push!(ids, [imodel, iq])
        end
        nid = size(ids, 1)

        expanded_T = make_expanded_transition(T, ids, qs)
        expanded_pi_i = make_expanded_pi_i(pi_i, ids)
        obs_list, log_emission = make_expanded_emission(ids, [r])
        estimated_state = viterbi(obs_list[1][1:my_nframe], expanded_T, expanded_pi_i, log_emission)
        
        logL[iq] = log(expanded_pi_i[estimated_state[1]]) + log_emission[estimated_state[1], 1]
        for iframe = 2:my_nframe
            i = estimated_state[iframe-1]
            j = estimated_state[iframe]
            logL[iq] += log(expanded_T[i, j]) + log_emission[j, iframe]
        end
    end
    push!(logLs, logL)
end

logLs_100frames = deepcopy(logLs);

In [34]:
my_nframe = 10

logLs = []
for pred_radius in pred_radii
    @load "data/result/test_radius_$(test_radius)/pred_radius_$(pred_radius)/sharpness_$(sharpness)/q_$(nq)/iq_$(iq_true)_noise_$(sigma_noise)_nframe_$(nframe).bson" params r
    logL = zeros(Float64, nq)
    for iq = 1:nq
        ids = []
        for imodel = 1:nmodel
            push!(ids, [imodel, iq])
        end
        nid = size(ids, 1)

        expanded_T = make_expanded_transition(T, ids, qs)
        expanded_pi_i = make_expanded_pi_i(pi_i, ids)
        obs_list, log_emission = make_expanded_emission(ids, [r])
        estimated_state = viterbi(obs_list[1][1:my_nframe], expanded_T, expanded_pi_i, log_emission)
        
        logL[iq] = log(expanded_pi_i[estimated_state[1]]) + log_emission[estimated_state[1], 1]
        for iframe = 2:my_nframe
            i = estimated_state[iframe-1]
            j = estimated_state[iframe]
            logL[iq] += log(expanded_T[i, j]) + log_emission[j, iframe]
        end
    end
    push!(logLs, logL)
end

logLs_10frames = deepcopy(logLs);

In [35]:
my_nframe = 1

logLs = []
for pred_radius in pred_radii
    @load "data/result/test_radius_$(test_radius)/pred_radius_$(pred_radius)/sharpness_$(sharpness)/q_$(nq)/iq_$(iq_true)_noise_$(sigma_noise)_nframe_$(nframe).bson" params r
    logL = zeros(Float64, nq)
    for iq = 1:nq
        ids = []
        for imodel = 1:nmodel
            push!(ids, [imodel, iq])
        end
        nid = size(ids, 1)

        expanded_T = make_expanded_transition(T, ids, qs)
        expanded_pi_i = make_expanded_pi_i(pi_i, ids)
        obs_list, log_emission = make_expanded_emission(ids, [r])
        estimated_state = viterbi(obs_list[1][1:my_nframe], expanded_T, expanded_pi_i, log_emission)
        
        logL[iq] = log(expanded_pi_i[estimated_state[1]]) + log_emission[estimated_state[1], 1]
        for iframe = 2:my_nframe
            i = estimated_state[iframe-1]
            j = estimated_state[iframe]
            logL[iq] += log(expanded_T[i, j]) + log_emission[j, iframe]
        end
    end
    push!(logLs, logL)
end

logLs_1frames = deepcopy(logLs);

In [39]:
diff_q = []
for i in 1:length(pred_radii)
    diff_q = []
    for iq = 1:nq
        push!(diff_q, quate_dist(qs[iq, :], qs[iq_true, :]))
    end
end

In [45]:
for i in 1:length(pred_radii)
    p = scatter(logLs_100frames[i], diff_q, 
        label=nothing, framestyle=:box,
        xlabel="log likelihood", ylabel="quaternion distance\nfrom the ground-truth",
        markersize=4, markerstrokewidth=0,
        size=(600, 400), dpi=300,
        left_margin=Plots.Measures.Length(:mm, 10.0),
        right_margin=Plots.Measures.Length(:mm, 10.0),
        bottom_margin=Plots.Measures.Length(:mm, 10.0))
    savefig(p, "fig05_$(pred_radii[i])A_100frames.png")

    p = scatter(logLs_10frames[i], diff_q, 
        label=nothing, framestyle=:box,
        xlabel="log likelihood", ylabel="quaternion distance\nfrom the ground-truth",
        markersize=4, markerstrokewidth=0,
        size=(600, 400), dpi=300,
        left_margin=Plots.Measures.Length(:mm, 10.0),
        right_margin=Plots.Measures.Length(:mm, 10.0),
        bottom_margin=Plots.Measures.Length(:mm, 10.0))
    savefig(p, "fig05_$(pred_radii[i])A_10frames.png")

    p = scatter(logLs_1frames[i], diff_q, 
        label=nothing, framestyle=:box,
        xlabel="log likelihood", ylabel="quaternion distance\nfrom the ground-truth",
        markersize=4, markerstrokewidth=0,
        size=(600, 400), dpi=300,
        left_margin=Plots.Measures.Length(:mm, 10.0),
        right_margin=Plots.Measures.Length(:mm, 10.0),
        bottom_margin=Plots.Measures.Length(:mm, 10.0))
    savefig(p, "fig05_$(pred_radii[i])A_1frames.png")
end